# 15 – Case Study: Sugar Reduction (Without Objective Measures)

**Learning Objectives:**
- Understand the challenges of evaluating dietary interventions without biomarkers
- Compare evaluation approaches for the Soft Drinks Industry Levy (SDIL)
- Critically assess the evidence hierarchy: purchase data, dietary surveys, modelled estimates
- Appreciate the role of uncertainty in policy evaluation
- Contrast the sugar and salt reduction programmes

---

## 1. Introduction: The Sugar Challenge

Unlike salt, sugar intake cannot be measured with a simple biomarker. This creates fundamental challenges for evaluating sugar reduction policies.

### Why No Biomarker?

- **Salt**: ~90% excreted in urine, easy to measure
- **Sugar**: Metabolised rapidly, no single biomarker captures total intake

### The UK Sugar Reduction Programme

| Policy | Year | Mechanism |
|--------|------|----------|
| Voluntary reformulation targets | 2016 | Industry pledges |
| Soft Drinks Industry Levy (SDIL) | 2018 | Fiscal measure |
| HFSS advertising restrictions | 2022+ | Marketing limits |

## 2. Setup

In [ ]:
# ============================================================
# Bootstrap cell
# ============================================================

import os
import sys
import pathlib
import subprocess

REPO_URL = "https://github.com/ggkuhnle/phn-epi.git"
REPO_DIR = "phn-epi"

cwd = pathlib.Path.cwd()

if (cwd / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd
elif (cwd.parent / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd.parent
else:
    repo_root = cwd / REPO_DIR
    if not repo_root.is_dir():
        print(f"Cloning repository from {REPO_URL} ...")
        subprocess.run(["git", "clone", REPO_URL, str(repo_root)], check=True)
    else:
        print(f"Using existing repository at {repo_root}")
    os.chdir(repo_root)
    repo_root = pathlib.Path.cwd()

scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

print(f"Repository root: {repo_root}")
print("Bootstrap completed successfully.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("Libraries loaded successfully.")

## 3. The Soft Drinks Industry Levy (SDIL)

The SDIL uses a **tiered structure**:

| Sugar content | Levy rate |
|---------------|----------|
| < 5g per 100ml | £0.00 |
| 5-8g per 100ml | £0.18 per litre |
| > 8g per 100ml | £0.24 per litre |

In [ ]:
# Product reformulation data (illustrative)
reformulation_data = pd.DataFrame({
    'Period': ['Pre-announcement (2015)', 'Post-announcement (2017)', 'Post-implementation (2019)'],
    'Mean sugar (g/100ml)': [4.4, 3.5, 2.9],
    'Products above higher threshold (%)': [52, 38, 15],
    'Products in zero-levy band (%)': [39, 49, 73]
})

print("Soft Drink Reformulation (Illustrative Data)")
print("=" * 70)
display(reformulation_data)

In [ ]:
# Visualise reformulation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean sugar content
ax = axes[0]
colors = ['#e74c3c', '#f39c12', '#27ae60']
bars = ax.bar(range(3), reformulation_data['Mean sugar (g/100ml)'], color=colors)
ax.set_xticks(range(3))
ax.set_xticklabels(['2015', '2017', '2019'])
ax.set_ylabel('Mean sugar (g/100ml)')
ax.set_title('Average Sugar Content in Soft Drinks')
ax.set_ylim(0, 5)
for i, v in enumerate(reformulation_data['Mean sugar (g/100ml)']):
    ax.text(i, v + 0.1, f'{v}g', ha='center', fontweight='bold')

# Levy band distribution
ax = axes[1]
width = 0.35
x = np.arange(3)
ax.bar(x - width/2, reformulation_data['Products above higher threshold (%)'], width, 
       label='High levy (>8g)', color='#e74c3c')
ax.bar(x + width/2, reformulation_data['Products in zero-levy band (%)'], width, 
       label='Zero levy (<5g)', color='#27ae60')
ax.set_xticks(x)
ax.set_xticklabels(['2015', '2017', '2019'])
ax.set_ylabel('Percentage of products')
ax.set_title('Distribution Across Levy Bands')
ax.legend()

plt.tight_layout()
plt.show()

print("\nKey finding: Substantial reformulation occurred, mostly before implementation.")

## 4. The Evidence Hierarchy for Sugar

Without a biomarker, we rely on indirect evidence:

1. **Product composition** - Did manufacturers reduce sugar? (High confidence)
2. **Purchase data** - Did households change buying? (Moderate confidence)
3. **Dietary surveys** - Did intake actually fall? (Lower confidence)
4. **Health outcomes** - Modelled estimates only (Uncertain)

In [ ]:
# Simulated purchase data trend
years = np.arange(2014, 2022)
sugar_purchased = np.array([155, 150, 140, 125, 105, 100, 98, 95])
counterfactual = np.array([155, 152, 150, 148, 146, 144, 142, 140])

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(years, sugar_purchased, 'o-', linewidth=2, markersize=8, color='steelblue', label='Observed')
ax.plot(years, counterfactual, 's--', linewidth=2, markersize=8, color='gray', label='Counterfactual')

ax.axvline(x=2016.25, color='orange', linestyle=':', linewidth=2, label='SDIL announced')
ax.axvline(x=2018.25, color='red', linestyle=':', linewidth=2, label='SDIL implemented')

ax.fill_between(years, sugar_purchased, counterfactual, alpha=0.3, color='green',
                where=years >= 2016, label='Estimated reduction')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Sugar from soft drinks (g/household/week)', fontsize=12)
ax.set_title('Household Purchases of Sugar from Soft Drinks', fontsize=14)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 5. The Counterfactual Problem

**What would have happened without the SDIL?**

Different assumptions → Different effect estimates

In [ ]:
# Different counterfactual scenarios
years = np.arange(2014, 2022)
observed = np.array([155, 150, 140, 125, 105, 100, 98, 95])

counterfactuals = {
    'Flat trend': np.array([155, 155, 155, 155, 155, 155, 155, 155]),
    'Slow decline (-2%/yr)': 155 * (0.98 ** np.arange(8)),
    'Moderate decline (-3%/yr)': 155 * (0.97 ** np.arange(8)),
}

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(years, observed, 'ko-', linewidth=3, markersize=10, label='Observed')

colors = ['#3498db', '#e74c3c', '#2ecc71']
for i, (name, cf) in enumerate(counterfactuals.items()):
    ax.plot(years, cf, '--', linewidth=2, color=colors[i], label=f'Counterfactual: {name}')

ax.axvline(x=2018.25, color='red', linestyle=':', linewidth=2)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Sugar from soft drinks (g/hh/week)', fontsize=12)
ax.set_title('The Counterfactual Problem', fontsize=13)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\nEstimated effect in 2021 depends on counterfactual assumption:")
for name, cf in counterfactuals.items():
    effect = cf[-1] - observed[-1]
    print(f"  {name}: {effect:.0f} g/hh/week reduction")

## 6. Comparing Salt and Sugar Evaluation

In [ ]:
comparison = pd.DataFrame({
    'Aspect': [
        'Objective biomarker',
        'Measurement confidence',
        'Causal pathway',
        'Time to health impact',
        'Overall confidence'
    ],
    'Salt': [
        '24-hour urinary sodium ✓',
        'High',
        'Well established (salt→BP→CVD)',
        'Years',
        'Moderate-high'
    ],
    'Sugar': [
        'None available ✗',
        'Low',
        'Complex (sugar→obesity→diseases)',
        'Decades',
        'Low'
    ]
})

print("Salt vs Sugar: Evaluation Comparison")
print("=" * 80)
display(comparison)

In [ ]:
# Evidence confidence visualisation
evidence_layers = [
    ('Product reformulation', 0.9),
    ('Household purchases', 0.7),
    ('Individual intake', 0.4),
    ('Weight/BMI change', 0.2),
    ('Disease outcomes', 0.1)
]

fig, ax = plt.subplots(figsize=(10, 5))

labels = [e[0] for e in evidence_layers]
confidence = [e[1] for e in evidence_layers]

colors = plt.cm.RdYlGn(np.array(confidence))
ax.barh(labels, confidence, color=colors)

ax.set_xlim(0, 1)
ax.set_xlabel('Confidence in evidence')
ax.set_title('SDIL Evaluation: Evidence Confidence by Outcome Level')

plt.tight_layout()
plt.show()

## 7. Discussion Questions

1. **The reformulation paradox**: If manufacturers reformulated before implementation, the levy raised less revenue. Is this success or failure?

2. **Substitution**: If people switch from taxed soft drinks to untaxed confectionery, has the policy failed?

3. **Evidence thresholds**: How much evidence should we require before implementing a dietary policy?

4. **Comparison with salt**: Why has salt reduction been more acclaimed? Is this justified?

## 8. Exercises

### Exercise 1: Critique a Published Evaluation

Read this abstract and identify limitations:

> "We used difference-in-differences analysis of household purchase data to estimate the effect of the SDIL. We found a 30% reduction in sugar from soft drinks attributable to the SDIL."

In [ ]:
# YOUR CRITIQUE HERE



### Exercise 2: Design an Evaluation

A new policy bans HFSS advertising on TV before 9pm. Design an evaluation:
- What outcomes would you measure?
- What data sources would you use?
- How would you construct a counterfactual?

In [ ]:
# YOUR DESIGN HERE



---

## Summary

- Sugar intake cannot be measured with a simple biomarker
- The SDIL achieved measurable reformulation and purchasing changes
- Health impact estimates are highly uncertain
- The counterfactual problem is fundamental
- Uncertainty should be communicated honestly

---

## References

- Pell D et al. (2021). Changes in soft drinks purchased by British households. *BMJ*.
- Bandy LK et al. (2020). Reductions in sugar sales from soft drinks. *BMC Medicine*.
- Scarborough P et al. (2020). Impact of the UK SDIL on childhood obesity. *Lancet Public Health*.